# 4.2 - Office finder mongo version

#### Encontramos tu oficina

In [1]:
# librerias

from pymongo import MongoClient

import pandas as pd

import folium   # esto es para hacer mapas

import re

In [2]:
# conexion a mongo

cliente=MongoClient()

db=cliente.companies

In [12]:
query=db.companies.find({'$and': [ {'$or': [{'category_code': 'games_video'},
                                      {'category_code': 'web'},
                                      {'category_code': 'mobile'},
                                      {'category_code': 'ecommerce'}]},
                           
                               {'founded_year': {'$gte': 1999}},
                               {'deadpooled_year': None},
                               {'number_of_employees': {'$ne': None}},
                               {'number_of_employees': {'$ne': 0}},
                               {'total_money_raised': {'$ne': None}}
                            
                           ]},
                 
                 {'name': 1, '_id':0, 'offices.latitude': 1, 'offices.longitude': 1,
                 'founded_year': 1, 'category_code':1, 'number_of_employees':1,
                 'total_money_raised':1
                 }
                 )




data=pd.json_normalize(data=query, record_path='offices', 
                       meta=['category_code', 'name', 'number_of_employees',
                            'founded_year', 'total_money_raised'])


data.head()

,latitude,longitude,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,games_video,Kyte,40,2006,$23.4M
4,37.480999,-122.173887,mobile,Jingle Networks,35,2005,$88.7M


In [15]:
# esto inserta una columna vacia en la posicion 2
data.insert(2, 'geo_loc', pd.Series(list(range(len(data)))))


# aqui relleno la columna con una lista de longitud y latitud
data['geo_loc']=data.apply(lambda x: [x['longitude'], x['latitude']], axis=1)


data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M
4,37.480999,-122.173887,"[-122.173887, 37.480999]",mobile,Jingle Networks,35,2005,$88.7M


In [23]:
def get_data():   # realizar query y traer datos
    
    query=db.companies.find({'$and': [ {'$or': [{'category_code': 'games_video'},
                                                {'category_code': 'software'},
                                                {'category_code': 'web'},
                                                {'category_code': 'mobile'},
                                                {'category_code': 'photo_video'},
                                                {'category_code': 'ecomerce'},
                                                {'category_code': 'search'},
                                                {'category_code': 'network_hosting'}]},
                                      
                                      {'founded_year': {'$gte': 1999}},
                                      {'offices': {'$ne': []}},
                                      {'offices.longitude': {'$ne': []}},
                                      {'offices.latitude': {'$ne': []}},
                                      {'offices.longitude': {'$ne': None}},
                                      {'offices.latitude': {'$ne': None}},
                                      {'deadpooled_year': None},
                                      {'number_of_employees': {'$ne': None}},
                                      {'total_money_raised': {'$ne': None}}]},

                            {'name': 1, '_id':0, 'offices.latitude': 1, 'offices.longitude': 1,
                             'founded_year': 1, 'category_code':1, 'number_of_employees':1,
                             'total_money_raised':1
                            })
    
    
    df=pd.json_normalize(data=query, record_path='offices',
                         meta=['category_code', 'name', 'number_of_employees',
                               'founded_year', 'total_money_raised'])
    
    
    # esto inserta una columna vacia en la posicion 2
    df.insert(2, 'geo_loc', pd.Series(list(range(len(df)))))
    
    # aqui relleno la columna con una lista de longitud y latitud
    df['geo_loc']=df.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
    
    return df

In [24]:
data=get_data()

data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M


In [16]:
def exchange(col):
    
    res=[]
    
    number=1
    
    symbol=1
    
    for e in col:
        
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€' : symbol=1.12
        
        n_col=re.findall('[0-9.]+', e)[0]

        
        res.append(int(float(n_col) * number * symbol))
        
    return res

In [17]:
exchange(['$28.3M'])

[28300000]

In [18]:
def classifying(date, employees):
    
    company_class=[]
    
    for i in range(len(date)):
        
        if date[i]>=2008 and employees[i]<100:
            company_class.append('StartUp')
        else:
            company_class.append('BigCorp')
            
    
    return company_class

In [25]:
data['n_total_money']=exchange(data.total_money_raised)

data['company_class']=classifying(data.founded_year, data.number_of_employees)


data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,BigCorp
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,BigCorp
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp


In [26]:
# insertar en mongo

#df.to_json('geoloc.json', orient='records', lines=True)

In [27]:
db.geoloc.insert_many(data.to_dict('records'))

In [28]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [31]:
# geoquery

list(db.geoloc.find({'geo_loc': {'$near': {'$geometry': {'type': 'Point',
                                                    'coordinates': data.geo_loc[0]},
                                     
                                      '$maxDistance': 1000  # 1000 metros
                                     }}}))

[{'_id': ObjectId('63074f9802673a606eb7c7d9'),
  'latitude': 34.090368,
  'longitude': -118.393064,
  'geo_loc': [-118.393064, 34.090368],
  'category_code': 'web',
  'name': 'Geni',
  'number_of_employees': 18,
  'founded_year': 2006,
  'total_money_raised': '$16.5M',
  'n_total_money': 16500000,
  'company_class': 'BigCorp'}]

In [32]:
# geolocalizacion

def geoquery(df):
    
    list_money_mean=[]
    
    sb_ratio=[]
    
    n_companies=[]
    
    for i in range(len(df)):
        
        q=list(db.geoloc.find({'geo_loc': {'$near': {'$geometry': {
                                     
                                'type': 'Point',
                                'coordinates': df.geo_loc[i]},
                                
                                '$maxDistance': 1000}}}))
        
        
        long_list=len(q)
        
        # media de dinero
        money_mean=sum([q[j]['n_total_money'] for j in range(long_list)]) / long_list
        
        list_money_mean.append(money_mean)
        
        # startup-bigcorp
        
        big=sum(q[k]['company_class'].count('BigCorp') 
                if q[k]['company_class'].count('BigCorp')!=0 
                else 1 for k in range(long_list))
        
        start=sum(q[k]['company_class'].count('StartUp')  for k in range(long_list))
        
        # si sb_ratio=1 entonces big=start, si sb_ratio<1 hay más big, si sb_ratio>1 hay mas start
        sb_ratio.append(start/big)
        
        
        # nº compañias
        n_companies.append(long_list)
        
        
    df['around_companies']=n_companies
    
    df['sb_ratio']=sb_ratio
    
    df['money_density']=list_money_mean
    
    return df

In [33]:
data=geoquery(data)

data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,around_companies,sb_ratio,money_density
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp,1,0.000000,1.650000e+07
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,BigCorp,8,0.375000,7.325250e+06
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp,2,0.000000,2.715000e+07
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,BigCorp,27,0.222222,1.105724e+07
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp,3,0.333333,8.733333e+06


In [34]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                    tiles='openstreetmap', 
                    zoom_start=zoom_start)
    
    
    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], 
                            popup=str(df.name[i]), 
                            radius=1, 
                            icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    return mapa

In [36]:
print('oficinas')

mapa=geomap(data.loc[:1500])

mapa

oficinas


In [38]:
print ('Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$')

df=data[(data.around_companies >= 10) & (data.sb_ratio > 0.45) & (data.money_density > 1e6)]


mapa=geomap(df)

mapa


Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$


In [39]:
print ('Compañias alrededor>=50, sb_ratio>0.25 y densidad monetaria>10.000.000$')

df=data[(data.around_companies >= 50) & (data.sb_ratio > 0.25) & (data.money_density > 1e7)]

mapa=geomap(df)

mapa


Compañias alrededor>=50, sb_ratio>0.25 y densidad monetaria>10.000.000$
